# Data preparation

In [1]:
!conda install --yes pytorch torchvision -c pytorch

Retrieving notices: ...working... done
Solving environment: done

# All requested packages already installed.



In [2]:
!pip install torchvision

In [3]:
import os
import numpy as np
np.random.seed(666)
import torchvision
if not os.path.isdir('data'):
    ! mkdir data

In [4]:
import torchvision.datasets as datasets
cifar10_trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=None)
cifar10_testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=None)

Files already downloaded and verified
Files already downloaded and verified


In [5]:
x_train = np.array([np.array(img[0])/255. for img in cifar10_trainset])
y_train = np.array([img[1] for img in cifar10_trainset])
x_test = np.array([np.array(img[0])/255. for img in cifar10_testset])
y_test = np.array([img[1] for img in cifar10_testset])

np.savez('data/cifar10', arr_0=x_train, arr_1=y_train, arr_2=x_test, arr_3=y_test)

In [6]:
# load images as queries

dataset = np.load('data/cifar10.npz')
x_train = dataset['arr_0']
y_train = dataset['arr_1']
x_test = dataset['arr_2']
y_test = dataset['arr_3']

print("training data shape: {}, {}, testing data shape: {}, {}".format(x_train.shape, y_train.shape, x_test.shape, y_test.shape))

training data shape: (50000, 32, 32, 3), (50000,), testing data shape: (10000, 32, 32, 3), (10000,)


# Start Blacklight detection

In [7]:
!pip install config

In [8]:
#!pip install logging

In [9]:
from probabilistic_fingerprint import *

In [10]:
# parameters for blacklight CIFAR10
window_size = 20
hash_kept = 50
roundto = 50
step_size = 1
workers = 5
threshold = 25

In [11]:
def get_tracker(query, window_size, hash_kept, roundto, step_size, workers):
    tracker = InputTracker(query, window_size, hash_kept, round=roundto, step_size=step_size, workers=workers)
    LOGGER.info("Blacklight detector created.")
    return tracker

In [12]:
# create a tracker as detector and detect malicious queries
tracker = get_tracker(x_train[0], window_size, hash_kept, roundto, step_size, workers)
match_list = []
id = 0
for query in x_test:
    match_num = tracker.add_img(query)
    match_list.append(match_num)
    if(match_num>threshold):
        LOGGER.info("Image: {}, max match: {}, attack_query: {}".format(id, match_num, match_num>threshold))
    id += 1

[18:06:36,05/05 /var/folders/93/bmyryj59741dyh2hjbrc92qw0000gn/T/ipykernel_34671/2128301390.py:3 INFO]: Blacklight detector created.


In [13]:
# calculate false positive rate
LOGGER.info("False positive rate on CiFAR-10 test dataset: {}".format(np.sum([1 for i in match_list if i > threshold])/len(x_test)))

[18:07:25,05/05 /var/folders/93/bmyryj59741dyh2hjbrc92qw0000gn/T/ipykernel_34671/3485651310.py:2 INFO]: False positive rate on CiFAR-10 test dataset: 0.0
